# 简历项目3：Ollama构建本地轻量级RAG系统
## 1. 项目简介

## 2. 数据集 & RAG系统
&emsp;&emsp;RAG基础阅读：[16种RAG方案一站式攻略](https://blog.csdn.net/l01011_/article/details/149929414)、[七大Agentic RAG框架技术解析](https://zhuanlan.zhihu.com/p/19229901774) <br>
&emsp;&emsp;[RAGEval数据集文献](https://arxiv.org/abs/2408.01262): Scenario Specific RAG Evaluation Dataset Generation Framework  <br>
&emsp;&emsp;[LightRAG文献](https://arxiv.org/abs/2410.05779)：Simple and Fast Retrieval-Augmented Generation <br>
&emsp;&emsp;本地配置：Ollama推理Qwen3-4B-Q8_0.gguf大模型和bge-m3-FP16.gguf嵌入模型，可以在魔搭社区上提前下载好再使用Modelfile配置一下，vllm推理bge-reranker重排序模型，在运行LightRAG之前在终端开启以上服务。<br>
&emsp;&emsp;扩展阅读：[大语言模型检索增强生成优化技术研究综述](https://kns.cnki.net/kcms2/article/abstract?v=VYuoLtjwl8MYmE9GqmY8xf1O2fecUU_GoKLRuNKU-AXEtLiOvAqt73fsdoc1L62K0gVUoqEl1fMU-PHeQi0fQ6PzmGkmjb8x483Ljy4UjeCjK8ZShmTxtWYBFmURFJ7SMPVCL0qVDVoAeS8RPrixwtQyrFdVs9hRv1CoClwDQ6U1lr5rIsd0xw==&uniplatform=NZKPT&language=CHS)、[检索增强生成技术研究综述](http://cea.ceaj.org/CN/10.3778/j.issn.1002-8331.2501-0061)、[检索增强生成在软件工程中的应用综述](https://kns.cnki.net/kcms2/article/abstract?v=VYuoLtjwl8NUINSF6UaZuKpxXpAOW_G5nb95hxqf2qltz5ibMQ0VuR90boRWvM6LrL-5K9nO9vmn_kF02NmRcRceWKeEjFeQZ688uocBK6oarvPseNdVa__ulkWomPWOeOtO0ZRDKgSfrqQyExy3hGEN1-oIJmy_F4R48dvLnBbHAOoQJWAfdg==&uniplatform=NZKPT&language=CHS)

## 3. LightRAG解读
<div align="center">
   <img src="./pics/rp3_rag_1.png" width="1000" />
</div>

&emsp;&emsp;参考阅读:[深度解析：LightRAG知识图谱实现于应用分析](https://mp.weixin.qq.com/s/scZ1WZRS3v1QBOYV_KYFTw)、

&emsp;&emsp;知识图谱是 LightRAG 区别于传统 RAG 系统的核心特性，它不仅将文档转换为向量，还构建了一个结构化的知识网络，让系统能够理解实体之间的关系，进行推理和发现。

### 3.1 文本文档预处理
&emsp;&emsp;lightrag.py中的 apipeline_enqueue_documents 异步文档入队管道函数，主要负责验证、去重、过滤和添加待处理文档到文档状态存储中。使用到的参数**input: str | list[str], ids: list[str] | None = None, file_paths: str | list[str] | None = None,track_id: str | None = None**。<br>
1. **标准化输入参数**：确保后续处理逻辑能够一致地处理列表格式的数据，并验证参数之间的逻辑关系，为后续的文档去重和ID管理做好准备。具体地，如果没有提供track_id或空字符串，则生成一个新的跟踪ID，前缀为"enqueue"。将单个字符串类型的input转换为列表格式，确保函数能处理单个或多个文档。此外，还会确保文件路径数量与输入文档数量匹配，**input参数**存储实际文本内容，**file_paths**记录文档来源，作为元数据。<br>
1. **文档去重和ID管理**：如果未提供ID，则自动生成MD5哈希作为文档ID，同时基于内容的唯一性自动去除重复文档，此外验证用户提供的ID是否唯一且数量匹配。具体地：
    1. ID管理策略：检查ID数量石佛与文档数量匹配，使用set(ids)检查是否唯一，ID必须是唯一的字符串，if len(ids) != len(set(ids))。**ids参数**作为每个输入文档的唯一ID，用于数据去重和索引，可以是多个ID。**track_id参数**跟踪一批文档的处理状态，用于查询处理进度，单个ID。
    1. 文档去重机制，**(1)内容级去重**使用清理后（清理文本以确保安全的UTF-8编码，通过移除或替换可能导致编码错误的问题字符。）的内容作为字典的键进行去重，相同内容只保留第一个出现的。**(2)存储级去重**使用filter_keys方法检查存储中是否已经存在相同ID的文档，只处理不存在的文档ID。内容去重和ID去重的双重去重机制，确保了系统中不会出现重复内容的文档，同时维护了文档ID的唯一性和系统的稳定性。
1. **文档状态初始化**：定义无文档内容字典变量new_docs={"id_1":{"status":"....",...},"id_2":{"status":"....",...}}嵌套字典，id_1为唯一标识字符。为新文档创建初始状态DocStatus.PENDING
    （class DocStatus(str, Enum):
    PENDING = "pending"
    PROCESSING = "processing"
    PREPROCESSED = "preprocessed"
    PROCESSED = "processed"
    FAILED = "failed"），其他键：content_summary: 从文档内容生成的摘要（前250个字符）；content_length: 文档内容的字符长度；created_at: 文档创建时间戳（ISO格式的UTC时间）；updated_at: 文档更新时间戳（ISO格式的UTC时间）；file_path: 文档的原始文件路径；track_id: 用于跟踪此批处理操作的ID。
1. **重复检测与过滤**：检查文档是否已存在于系统中，过滤掉已处理过的文档，避免重复处理，记录被忽略的文档ID并输出警告。
1. **数据持久化**：将文档内容存储到full_docs存储中，将文档状态存储到doc_status存储中。立即执行index_done_callback()确保数据持久化。
1. **错误处理与日志**：提供详细的日志记录，包括处理进度和警告信息。对于重复文档他提供清晰的警告信息，支持跟踪ID用于监控处理状态。

### 3.2 知识图谱构建
&emsp;&emsp;lightrag.py中的apipeline_process_enqueue_documents函数是LightRAG的文档处理管道，负责将待处理文档转换为知识图谱。这个函数本质上是一个生产者-消费者模式的处理管道，负责将原始文档转换为结构化知识图谱数据。主要功能如下：<br>
1. **处理排队的文档**：使用分布式锁确保同一时间只有一个进程处理队列，收集三种状态的文档(processing,failed,pending)，如果已有进程在处理，设置request_pending标志并返回。
1. **文档状态管理**：合并待处理文档，初始化管道状态（作业名称、开始时间、进度等）。
1. **数据一致性验证**：。
1. **多阶段处理流程**：。
    1. **阶段1：文档分块与存储**<br>
    两种分块策略类型：<br>
    1、基于Token的分块（默认），按照token数量分割文档，默认块的大小为1200tokens，重叠大小为100tokens。<br>
    2、基于字符的分块，按照指定字符分割文档，支持仅按字符分割或结合token大小限制。**参数说明**：<font color='red'>**split_by_character: str | None**</font>，类型: str | None，说明: 指定用来分割文本的字符，如果不是 None，会先按照这个字符分割文本，然后再根据 token 限制进一步分割（除非设置了 split_by_character_only=True）；<font color='red'>**split_by_character_only: bool**</font>，类型: bool，说明:如果为 True，仅按 split_by_character 指定的字符分割，不考虑 token 限制，如果为 False，会在按字符分割后，再按 token 大小限制进行二次分割。<br>
    1. **阶段2：实体关系提取**
    1. **阶段3：合并到知识图谱**
1. **错误处理和取消支持**
1. **资源管理**
1. **进度追踪**
1. **处理流程总结**<br>
&emsp;待处理文档 → 文档分块 → 存储文本块/向量块 → 提取实体关系 → 合并到知识图谱 → 更新文档状态<br>
&emsp;&emsp;↓<br>
&emsp;并发控制（信号量）<br>
&emsp;&emsp;↓<br>
&emsp;错误处理与状态管理<br>
&emsp;&emsp;↓<br>
&emsp;循环检查新文档<br>


## 4. 面试问答
1. 有没有了解过GraphRAG？
1. 